In [1]:
import os
import pandas as pd
import numpy as np
import librosa
from librosa.effects import pitch_shift, time_stretch
import librosa.display
import soundfile as sf
import IPython.display as ipd
#import torchaudio
from random import randint, randrange
import matplotlib as mpl
import matplotlib.pyplot as plt
import urllib
import random
import shutil
import chirplet as ch
#%matplotlib notebook

Crear un dataframe con 2000 etiquetas de 'not validated' extraidas de forma aleatoria

In [16]:
df = pd.read_csv('../sound_annotation_files_splited/not_validated/not_validated.csv')

In [17]:
index_a = [] # crea una lista para almacenar los numero random sin repetir.
count_a = 0 # almacena las veces que se agrega un numero a la lista.

numero_random_a = randrange(0,len(df)) 

# se repite hasta que el tamaño de la lista no llegue a 600 (para el caso que solo quiera 600 numeros).
while count_a < 2000:
    numero_random_a = randrange(0,len(df))
    if numero_random_a in index_a: # verifica si el número random ya se encuentra en la lista
        pass
    else:
        index_a.append(numero_random_a) # agrega el número a la lista
        count_a = count_a + 1 # suma a cada vuelta

df = df.iloc[index_a]

In [18]:
len(df)

2000

In [19]:
df_Angachilla = df[df.site=='Angachilla']
len(df_Angachilla)

592

In [20]:
df_Miraflores = df[df.site=='Miraflores']
len(df_Miraflores)

893

In [21]:
df_PUEBosque = df[df.site=='Parque Urbano El Bosque']
len(df_PUEBosque)

515

In [22]:
df_Angachilla.to_csv('../sound_annotation_files_splited/not_validated/592Angachilla_toPOSP_pw.csv')
df_Miraflores.to_csv('../sound_annotation_files_splited/not_validated/893Miraflores_toPOSP_pw.csv')
df_PUEBosque.to_csv('../sound_annotation_files_splited/not_validated/515PUEBosque_toPOSP_pw.csv')

In [23]:
recording_dir = '../../exp1/arbimon2-cnn/recordings/' # Folder holding recordings
call_dir = '../data_aug_audio_samples/POSP/pw/xeno-canto_wet/' # Folder with external audio call-shapes 
new_samples_stft = '../data_aug_new_samples/STFT/POSP/pw/' # Folder where augmented data (spectrograms) will be stored
new_samples_fct = '../data_aug_new_samples/FCT/POSP/pw/'

In [24]:
# Read csv:
df_Angachilla = pd.read_csv('../sound_annotation_files_splited/not_validated/592Angachilla_toPOSP_pw.csv')
df_Miraflores = pd.read_csv('../sound_annotation_files_splited/not_validated/893Miraflores_toPOSP_pw.csv')
df_PUEBosque = pd.read_csv('../sound_annotation_files_splited/not_validated/515PUEBosque_toPOSP_pw.csv')

In [25]:
filez = sorted(os.listdir(call_dir))
filez = [f for f in filez if f.endswith('.wav')]
filez

['pw_01.wav',
 'pw_02.wav',
 'pw_03.wav',
 'pw_04.wav',
 'pw_05.wav',
 'pw_06.wav',
 'pw_07.wav',
 'pw_08.wav',
 'pw_09.wav',
 'pw_10.wav',
 'pw_11.wav',
 'pw_12.wav',
 'pw_13.wav',
 'pw_14.wav',
 'pw_15.wav',
 'pw_16.wav',
 'pw_17.wav',
 'pw_18.wav',
 'pw_19.wav',
 'pw_20.wav',
 'pw_21.wav',
 'pw_22.wav',
 'pw_23.wav',
 'pw_24.wav',
 'pw_25.wav',
 'pw_26.wav',
 'pw_27.wav',
 'pw_28.wav',
 'pw_29.wav',
 'pw_30.wav',
 'pw_31.wav',
 'pw_32.wav',
 'pw_33.wav',
 'pw_34.wav']

In [26]:
def augmentation(y, sr):
    #y : audio data from external call
    #sr: sample rate from librosa.load
    call_1 = y #identical output
    call_2 = y*0.5 #energy reduction
    call_3 = pitch_shift(call_2, sr, 1, bins_per_octave=12, res_type='kaiser_best') #1 semitone up
    call_4 = pitch_shift(y, sr, 1, bins_per_octave=12, res_type='kaiser_best') #1 semitone up
    call_5 = pitch_shift(call_2, sr, -1, bins_per_octave=12, res_type='kaiser_best') #1 semitone down
    call_6 = pitch_shift(y, sr, -1, bins_per_octave=12, res_type='kaiser_best') #1 semitone down
    call_7 = time_stretch(call_2, 0.9) #signal is slowed down
    call_8 = time_stretch(y, 0.8) #signal is slowed down
    call_9 = time_stretch(call_2, 1.1) #signal is sped up
    call_10 = time_stretch(y, 1.2) #signal is sped up
    call_11 = pitch_shift(call_7, sr, -1, bins_per_octave=12, res_type='kaiser_best') #slow and down
    call_12 = pitch_shift(call_9, sr, 1, bins_per_octave=12, res_type='kaiser_best') #speed and up 
    # call_2 (power reduction) is used for when the parameter setting is lower #
    data_augm = [call_1, call_2, call_3, call_4, call_5, call_6,
                 call_7, call_8, call_9, call_10, call_11, call_12]
    return(data_augm)

In [27]:
# Download audio from Arbimon:

### not validated ###
k = 0
for i in list(set(df_Angachilla.url)): # loop over recordings
    tmp = df_Angachilla[df_Angachilla.url==i]
    audio_filename = tmp.iloc[0].url.split('download/')[1]
    if not os.path.exists(recording_dir+audio_filename):
        urllib.request.urlretrieve(tmp.iloc[0].url, recording_dir+audio_filename)
        k = k+1
        print(k)
print('Audios of not-validated OK')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
Audios of not-validated OK


In [28]:
# Download audio from Arbimon:

### not validated ###
k = 0
for i in list(set(df_Miraflores.url)): # loop over recordings
    tmp = df_Miraflores[df_Miraflores.url==i]
    audio_filename = tmp.iloc[0].url.split('download/')[1]
    if not os.path.exists(recording_dir+audio_filename):
        urllib.request.urlretrieve(tmp.iloc[0].url, recording_dir+audio_filename)
        k = k+1
        print(k)
print('Audios of not-validated OK')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [29]:
# Download audio from Arbimon:

### not validated ###
k = 0
for i in list(set(df_PUEBosque.url)): # loop over recordings
    tmp = df_PUEBosque[df_PUEBosque.url==i]
    audio_filename = tmp.iloc[0].url.split('download/')[1]
    if not os.path.exists(recording_dir+audio_filename):
        urllib.request.urlretrieve(tmp.iloc[0].url, recording_dir+audio_filename)
        k = k+1
        print(k)
print('Audios of not-validated OK')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
Audios of not-validated OK


In [36]:
%%time
sample_time = 2
sampling_rate = 44100
missed = 0

C = [] #list with 34 arrays. Shape:12x88200
for n, f in enumerate(filez):
    call, rate = librosa.load(call_dir+f, sr=sampling_rate)
    call = call[0:sampling_rate*sample_time]
    call_augm = augmentation(call, rate)
    C.append(call_augm)
C = np.stack(C)
print(C.shape)
       
ii = 0
n_total_aug = len(filez)*len(C[0]) #408 total audio calls (34*12)
print(n_total_aug)
B = np.zeros([n_total_aug, len(C[0][0])])
print(B.shape)
for i in list(set(df_Miraflores.url)): # loop over recordings
    tmp = df_Miraflores[df_Miraflores.url==i]
    audio_filename = i.split('download/')[1]
    for c in range(len(tmp)): # loop over labels in the same recording
        sound_start, sound_end = [tmp.iloc[c]['x1'], tmp.iloc[c]['x2']]
        shft = ((sound_end-sound_start)-sample_time)/2
        start_sample = round(sampling_rate*(sound_start+shft))
        end_sample = start_sample+round(sampling_rate*sample_time)
        try:
            audio_data, rate = librosa.load(recording_dir+audio_filename, sr=sampling_rate)
            background = audio_data[int(start_sample): int(end_sample)]
            B[ii] = background
            ii = ii + 1
        except:
            print('ERROR\n', audio_filename)
            missed = missed + 1
            continue
    if ii == n_total_aug:
        break
B = np.split(B,len(filez))

(34, 12)
408
(408, 88200)
ERROR
 47816416
ERROR
 47808933
ERROR
 46845214
ERROR
 39787233
ERROR
 39815742
CPU times: user 32.1 s, sys: 382 ms, total: 32.5 s
Wall time: 32.6 s


In [37]:
%%time
# Create mixed spectrogram STFT:
for i in range(len(filez)):
    for j in range(len(B[0])):
        C_temp = C[i][j]
        B_temp = B[i][j]
        if len(C_temp) < len(B_temp): # correct shape product of time stretch (when it sped up)
            dif = len(B_temp) - len(C_temp)
            C_temp = np.append(C_temp, np.zeros([dif]))
        if len(C_temp) > len(B_temp): # correct shape product of time stretch (when it sped down)
            C_temp = C_temp[0:len(B_temp)]
        mix = B_temp*0.8 + C_temp
        sf.write('../data_aug_new_audios/POSP/pw/AudioAugm_POSP_pw_'+'Miraflores_'+str(i)+'_'+str(j)+'.wav', 
                 data = mix, 
                 samplerate = sampling_rate, 
                 format = 'wav', 
                 subtype = 'PCM_16')
        spec = librosa.feature.melspectrogram(y=mix, sr=sampling_rate, n_fft=2048, n_mels=128,
                                               hop_length=512, win_length=1024)
        spec = librosa.power_to_db(spec, ref=np.max)
        dpi=100
        fig = plt.figure(num=None, figsize=(300/dpi, 300/dpi), dpi=dpi)
        ax = plt.axes()
        ax.set_axis_off()
        librosa.display.specshow(spec)
        plt.savefig(new_samples_stft+'SpecAugm_POSP_pw_'+'Miraflores_'+str(i)+'_'+str(j)+'.png', bbox_inches='tight', transparent=True, pad_inches=0.0)
        plt.close()

CPU times: user 1min 48s, sys: 1.88 s, total: 1min 50s
Wall time: 33.2 s


In [ ]:
%%time
# Create mixed spectrogram CHIRPLET:

chirps = ch.FCT( duration_longest_chirplet=1,
                 num_octaves=5,
                 num_chirps_by_octave=10,
                 polynome_degree=0,
                 end_smoothing=0.001,
                 sample_rate=sampling_rate
               )

for i in range(len(filez)):
    for j in range(len(B[0])):
        C_temp = C[i][j]
        B_temp = B[i][j]
        if len(C_temp) < len(B_temp): # correct shape product of time stretch (when it sped up)
            dif = len(B_temp) - len(C_temp)
            C_temp = np.append(C_temp, np.zeros([dif]))
        if len(C_temp) > len(B_temp): # correct shape product of time stretch (when it sped down)
            C_temp = C_temp[0:len(B_temp)]
        mix = B_temp*0.7 + C_temp*1.4
        fct = chirps.compute(mix)
        dpi=100
        fig = plt.figure(num=None, figsize=(300/dpi, 300/dpi), dpi=dpi)
        ax = plt.axes()
        ax.set_axis_off()
        librosa.display.specshow(librosa.power_to_db(fct[::-1], ref=np.max),cmap='cividis')
        plt.savefig(new_samples_fct+'SpecAugm_POSP_pw_'+str(i)+'_'+str(j)+'.png', bbox_inches='tight', transparent=True, pad_inches=0.0)
        plt.close()

Ok !!!

In [ ]:
# Create mixed spectrogram TO OVERLEAF!!!:
S = []
data_audio = []
for i in range(len(filez)):
    for j in range(len(B[0])):
        C_temp = C[i][j]
        B_temp = B[i][j]
        if len(C_temp) < len(B_temp): # correct shape product of time stretch (when it sped up)
            dif = len(B_temp) - len(C_temp)
            C_temp = np.append(C_temp, np.zeros([dif]))
        if len(C_temp) > len(B_temp): # correct shape product of time stretch (when it sped down)
            C_temp = C_temp[0:len(B_temp)]
        mix = B_temp*0.7 + C_temp*1.4
        data_audio.append(mix)
        spec = librosa.feature.melspectrogram(y=mix, sr=sampling_rate, n_fft=2048, n_mels=128,
                                               hop_length=512, win_length=1024, fmin=512, fmax=8192)
        spec = librosa.power_to_db(spec, ref=np.max)
        S.append(spec)

### PLOT FIGURE TO OVERLEAF

In [ ]:
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
props = dict(boxstyle='round', facecolor='steelblue', alpha=0.7, edgecolor='k')
textlabel=['no fx', 'energy reduction', '1 semitone up', '2 semitone up', '1 semitone down', 
           '2 semitone down', 'slowed down', 'more slowed down',
           'sped up', 'more sped up', 'slow and down', 'speed and up']
fig, axlist = plt.subplots(3, 4, sharex=True, sharey=True, figsize=(18, 10))
for i in range(12):
    ax = plt.subplot(3,4,i+1)
    img = librosa.display.specshow(S[i+48],
                                 ax=ax,
                                 x_axis='time',
                                 y_axis='mel',
                                 fmin=512,
                                 fmax=8192,
                                 sr=48000,
                                 cmap='cividis'
                                )
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.tick_params(axis='both', which='major', labelsize=8)
    ax.tick_params(axis='both', which='minor', labelsize=8)
    ax.text(0.03, 0.97, textlabel[i], c='w',
            transform=ax.transAxes, 
            fontsize=11, 
            verticalalignment='top', 
            bbox=props)

fig.supxlabel('Time (s)', fontsize=12)
fig.supylabel('Frequency (Hz)', fontsize=12)
cax,kw = mpl.colorbar.make_axes([ax for ax in axlist.flat], 
                                orientation='vertical',
                                location='right',
                                fraction=0.02, 
                                pad=0.03)
plt.colorbar(img, cax=cax, **kw, format="%.f dB").set_label(label='',size=10)

plt.tight_layout(rect=(0.01, 0, 0.88, 1))
plt.show()
plt.savefig('DATA_AUG_pw_4.png',facecolor='w', format='png', dpi=600)